In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import (col, monotonically_increasing_id)
from notebookUtils import readCsvAsDataframe

In [9]:
spark = SparkSession.builder.appName("factsales2").getOrCreate()

In [10]:
fact_sales = readCsvAsDataframe(spark,"fact_sales_data_v2.csv")

### Create Store Region Dimension

In [14]:
df_store = fact_sales.select([
    col('StoreRegion').alias('Region'),
    col('StoreName').alias('Name'),
    col('StoreType').alias('Type')
]).distinct()

In [24]:
df_store2 = df_store.withColumn("StoreKey",monotonically_increasing_id())

In [25]:
df_store_na = spark.createDataFrame([
    ("N/A","N/A","N/A",-1)
],["Region","Name","Type", "StoreKey"])

In [26]:
dim_store = df_store2.unionAll(df_store_na)

In [27]:
dim_store.write.format("parquet").mode("overwrite").saveAsTable(name="dim_store")

In [28]:
df_dim_store = spark.read.parquet("spark-warehouse/dim_store")

In [31]:
df_dim_store.show(30)

[Row(Region='North', Name='StoreY', Type='Outlet', StoreKey=0),
 Row(Region='South', Name='StoreY', Type='Retail', StoreKey=1),
 Row(Region='East', Name='StoreX', Type='Retail', StoreKey=2),
 Row(Region='North', Name='StoreX', Type='Outlet', StoreKey=3),
 Row(Region='North', Name='StoreZ', Type='Retail', StoreKey=4),
 Row(Region='South', Name='StoreZ', Type='Franchise', StoreKey=5),
 Row(Region='South', Name='StoreZ', Type='Retail', StoreKey=6),
 Row(Region='East', Name='StoreX', Type='Franchise', StoreKey=7),
 Row(Region='South', Name='StoreX', Type='Outlet', StoreKey=8),
 Row(Region='East', Name='StoreZ', Type='Franchise', StoreKey=9),
 Row(Region='West', Name='StoreX', Type='Retail', StoreKey=10),
 Row(Region='North', Name='StoreY', Type='Retail', StoreKey=11),
 Row(Region='West', Name='StoreY', Type='Outlet', StoreKey=12),
 Row(Region='North', Name='StoreX', Type='Franchise', StoreKey=13),
 Row(Region='West', Name='StoreX', Type='Outlet', StoreKey=14),
 Row(Region='South', Name='St